In [1]:
import glob
invasive_dirs = [
    r'D:\VeligerData\invasive',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Umbonal',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Umbonal', 
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Zebra D-Hinge',
    ]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1 Image1\Baylor Preserved Zebra Umbo 1 Image1\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Baylor Preserved Zebra Umbo 1a Image1\Baylor Preserved Zebra Umbo 1a Image1\Sorted Images\Not',
    r'D:\VeligerData\USGS Labled Zebra\USGS Labled Zebra\Preserved Zebra D-Hinge 1 Baylor\Preserved Zebra D-Hinge 1 Baylor\Sorted Images\Not',
    r'D:\VeligerData\noninvasive',
    ]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    ]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))
    
import glob
from PIL import Image

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

In [ ]:

import os
import numpy as np
import tensorflow as tf
from PIL import Image

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in image_files:
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')

# Split the data into train and test sets
from sklearn.utils import shuffle

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names
X_train, X_test, Y_train, Y_test = train_test_split(train_images, train_labels_categorical, test_size=0.05, random_state=42)

In [ ]:
np.shape(X_train)

In [ ]:
import numpy as np
import pandas as pd
from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(40, 40, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3)))
model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.Dense(10))
model.add(layers.Dense(3, activation="softmax"))
model.summary()

In [ ]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
invasive_dirs = [
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Zebra Pediveliger Image1a\Zebra Pediveligers',
    r'D:\VeligerData\Baylor 2022-03-21_2\Veligers',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Pedi-Zebra Veligers',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Preserved Zebra Ped 1a',
]

# Non-Invasive category
non_invasive_dirs = [
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Not Veligers\O1',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Not Veligers\Z-P',
    r'D:\VeligerData\Baylor 2022-03-21_2\NonVeligers\Images_001',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1 To Baylor\Preserved Zebra Ped 1 To Baylor\Sorted Images\Not',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Zebra Ped 1a To Baylor\Preserved Zebra Ped 1a To Baylor\Sorted Images\Not',
]

# Ostracod category
ostracod_dirs = [
    r'D:\VeligerData\Ostracod Day 2 Image12 Short To Baylor\Ostracod Day 2 Image12 To Baylor\Sorted Images\Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1 To Baylor\Preserved Ostracods 1 To Baylor\Sorted Images\Preserve Ostracods',
    r'D:\VeligerData\Ostracod vs Pedi-Veliger Examples\Ostracod vs Pedi-Veliger Examples\Preserved Ostracods 1a To Baylor\Preserved Ostracods 1a To Baylor\Sorted Images\Preserved Ostracods 1a',
    r'D:\VeligerData\To Baylor 2023-01-30\To Baylor 2023-01-30\Ostracod Image1\Ostracods1',
    r'D:\VeligerData\Ostracods Day 2 Image1 To Baylor\Ostracods',
    ]

# List to store subdirectories
invasive_subdirs = []
non_invasive_subdirs = []
ostracod_subdirs = []

# Collect subdirectories in the invasive category
for invasive_dir in invasive_dirs:
    invasive_subdirs.extend(glob.glob(invasive_dir))

# Collect subdirectories in the non-invasive category
for non_invasive_dir in non_invasive_dirs:
    non_invasive_subdirs.extend(glob.glob(non_invasive_dir))
    
for ostracod_dir in ostracod_dirs:
    ostracod_subdirs.extend(glob.glob(ostracod_dir))

In [ ]:
import glob
from PIL import Image

# List of directories in x
x = non_invasive_subdirs

# List to store subdirectories
y1 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y1.extend(subdirectories)

import glob

# List of directories in x
x = invasive_subdirs

# List to store subdirectories
y2 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y2.extend(subdirectories)

    
import glob

# List of directories in x
x = ostracod_subdirs

# List to store subdirectories
y3 = []

# Read subdirectories for each directory in x
for directory in x:
    subdirectories = glob.glob(directory + r'\*')
    y3.extend(subdirectories)

    

    
import os
import numpy as np
import tensorflow as tf
from PIL import Image

def preprocess_images(y1, label_num, target_size=(40, 40)):
    # List to store image files
    image_files = []
    # List to store labels
    labels = []

    # Retrieve image files and create labels for each directory in y1
    for directory in y1:
        for root, dirs, files in os.walk(directory):
            for file in files:
                # Check if the file has an image extension
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    # Add the file path to the image_files list
                    image_files.append(os.path.join(root, file))
                    # Add the label to the labels list
                    

    # List to store preprocessed images
    images = []

    # Preprocess each image
    for file in image_files:
        try:
            # Read the image using PIL
            image = Image.open(file)
            # Resize the image using tf.image.resize_with_crop_or_pad()
            image = tf.image.resize_with_crop_or_pad(
                tf.keras.preprocessing.image.img_to_array(image),
                target_size[0],
                target_size[1]
            )
            # Normalize the image pixels for ML training
            image = image / 255.0
            # Add the preprocessed image to the images list
            images.append(image)
            labels.append(label_num)
        except Exception as e:
            print(f"Error processing image: {file}. Skipping...")
            continue

    # Convert the images and labels lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    return images, labels


X0,Y0 = preprocess_images(y1, label_num = 0)
X1,Y1 = preprocess_images(y2, label_num = 1)
X2,Y2 = preprocess_images(y3, label_num = 2)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Concatenate the image arrays
train_images = np.concatenate((X0, X1, X2), axis=0)

# Combine the label arrays
train_labels = np.concatenate((Y0, Y1, Y2), axis=0)

# Convert the labels to categorical
train_labels_categorical = to_categorical(train_labels)

# Set the dtype of train_images to float32
train_images = train_images.astype('float32')


# Split the data into train and test sets
from sklearn.utils import shuffle

# Assuming you have the original train_images and train_labels_categorical

# Shuffle the data
train_images, train_labels_categorical = shuffle(train_images, train_labels_categorical)

# Now the shuffled data is assigned to the same variable names

X_train, X_test1, Y_train, Y_test1 = train_test_split(train_images, train_labels_categorical, test_size=0.01, random_state=42)


In [ ]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("premodel.h5")

In [ ]:
np.shape(X_train)

In [ ]:
class_weights = np.zeros(3)

unique_labels, counts = np.unique(train_labels, return_counts=True)

total_samples = np.sum(counts)

for i, label in enumerate(unique_labels):
    class_weights[label] = total_samples / (len(unique_labels) * counts[i])

print("Class weights:", class_weights)

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics='accuracy')
class_weights = [1,1,1]
model.fit(X_train, Y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

In [ ]:
pred = model.predict(X_test)
p = np.argmax(pred, axis=1)
res = p
        
test = []
for i in Y_test:
    if(i[0]==1):
        test.append(0)
    elif(i[1]==1):
        test.append(1)
    else:
        test.append(2)
        
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(test,np.argmax(Y_test, axis=1))
print("Accuracy:", accuracy)

In [ ]:
# Define the image path
from PIL import Image
image_path = r"D:\VeligerData\Baylor 2022-03-21_2\Veligers\Object_031\Image_009.png"
# Read the image using PIL
image = Image.open(image_path)
# Define the target size for resizing
target_size = (40, 40)
# Resize the image using TensorFlow's resize_with_crop_or_pad function
resized_image = tf.image.resize_with_crop_or_pad( tf.keras.preprocessing.image.img_to_array(image), target_size[0],   target_size[1]
)
# Expand the dimensions to match the model's input shape
expanded_image = tf.expand_dims(resized_image, axis=0)
# Evaluate the TensorFlow tensor and obtain the NumPy array
resized_array = tf.keras.backend.eval(expanded_image)
normalized_image = resized_array / 255.0
x = model.predict(normalized_image)
x

In [ ]:
from tensorflow.keras.models import load_model
# Assuming you have a Keras model named "model"
model.save("single_image_model.h5")